In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import time, logging, gc
from sklearn.preprocessing import RobustScaler
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras import *
from tensorflow.keras.callbacks import *

In [ ]:
train = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
test = pd.read_csv('../input/ventilator-pressure-prediction/test.csv')

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
# copy from https://www.kaggle.com/hijest/gaps-features-tf-lstm-resnet-like-ff/notebook?scriptVersionId=77500256&cellId=3
def add_features(df): #CV 0.1579
    
    df['u_in'] = np.sqrt(2*df['u_in'])
    
    df['area'] = df['time_step'] * df['u_in']
    df['area'] = df.groupby('breath_id')['area'].cumsum()
    df['state'] = np.array([1 if x>0 else 0 for x in df['u_in']]) - df['u_out']
    
    df['exhale'] = df.groupby('breath_id')['u_out'].cumsum()
        
    df['delta_time'] = df.groupby('breath_id')['time_step'].diff().fillna(0)
    df['delta_u_in'] = df.groupby(df['breath_id'])['u_in'].diff().fillna(0).reset_index(level=0,drop=True)     
            
    df['u_in_1st_derivative'] = (df['delta_u_in'] /df['delta_time']).fillna(0)
    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
    
    df['u_in_lag1'] = df.groupby('breath_id')['u_in'].shift(1).fillna(0).reset_index(level=0,drop=True)   
    #df['u_in_lag2'] = df.groupby('breath_id')['u_in'].shift(2).fillna(0).reset_index(level=0,drop=True)
    df['u_in_lag3'] = df.groupby('breath_id')['u_in'].shift(3).fillna(0).reset_index(level=0,drop=True)
    df['u_in_back1'] = df.groupby('breath_id')['u_in'].shift(-1).fillna(0).reset_index(level=0,drop=True)   
    #df['u_in_lag2'] = df.groupby('breath_id')['u_in'].shift(2).fillna(0).reset_index(level=0,drop=True)
    df['u_in_back3'] = df.groupby('breath_id')['u_in'].shift(-3).fillna(0).reset_index(level=0,drop=True)
    #df['u_in_lag6'] = df.groupby('breath_id')['u_in'].shift(6).fillna(0).reset_index(level=0,drop=True)
    df['u_in_diff3'] = df['u_in'] - df['u_in_lag3']
#     df['u_out_diff3'] = df['u_out'] - df['u_out_lag3']
    df['u_in_diff1'] = df['u_in'] - df['u_in_lag1']
#     df['u_out_diff4'] = df['u_out'] - df['u_out_lag4']
    
    df['volume_mean']=((df['u_in'] + df['u_in'].shift(1).fillna(0))/2 * df['time_step'].diff().fillna(0)).clip(0,)
      
    df['volume_mean_10']=df.groupby(df['breath_id'])['volume_mean'].rolling(window=10, min_periods=1).sum().reset_index(level=0,drop=True)
    df['volume_mean_20']=df.groupby(df['breath_id'])['volume_mean'].rolling(window=20, min_periods=1).sum().reset_index(level=0,drop=True)
       
    df['volume_in_cumsum']=df.groupby('breath_id')['volume_mean'].cumsum()     
        
    df['_volume'] = df['volume_in_cumsum'] * (1 - df['u_out'])
    df['tidal_volume']=df.groupby(df['breath_id'])['_volume'].transform('max')
    df['volume_out'] = -(df['tidal_volume']/ np.log(df['exhale'])).replace(np.inf,0)
    
    df['volume_out_cumsum']=df.groupby('breath_id')['volume_out'].cumsum() 
    
    #df['u_in_rol_q0.1'] = df.groupby(df['breath_id'])['u_in'].rolling(window=10, min_periods=1).quantile(0.1).reset_index(level=0,drop=True)
    df['u_in_rol_q0.25'] = df.groupby(df['breath_id'])['u_in'].rolling(window=10, min_periods=1).quantile(0.25).reset_index(level=0,drop=True)
    #df['u_in_rol_q0.5'] = df.groupby(df['breath_id'])['u_in'].rolling(window=10, min_periods=1).quantile(0.5).reset_index(level=0,drop=True)
    df['u_in_rol_q0.75'] = df.groupby(df['breath_id'])['u_in'].rolling(window=10, min_periods=1).quantile(0.75).reset_index(level=0,drop=True)
    #df['u_in_rol_q0.9'] = df.groupby(df['breath_id'])['u_in'].rolling(window=10, min_periods=1).quantile(0.9).reset_index(level=0,drop=True)  

    df['PdV'] = df['delta_time']*df['volume_mean'] 
    df['PdV'] = df.groupby(df['breath_id'])['PdV'].cumsum()
    df['PdV'] = df['PdV'] / df['C'] #см вд столба
    
    df['PdV-'] = df['delta_time']*df['volume_out']
    df['PdV-'] = df.groupby(df['breath_id'])['PdV-'].cumsum()
    df['PdV-'] = df['PdV-'] / df['C'] #см вд столба
    
    df['dP'] = df['PdV'] + (df['volume_mean'] * df['R']) /1000 #см вд столба
    
    df['RC'] = (df['R']*df['C']).astype(str)
    df['R'] = df['R'].astype(str)
    df['C'] = df['C'].astype(str)
    df = pd.get_dummies(df)

    
    df = df.fillna(0)
    
    return df

#
train = add_features(train)
test = add_features(test)
#-------------cell------------------
targets = train[['pressure']].to_numpy().reshape(-1, 80)
train.drop(['id','breath_id','time_step','pressure'], axis=1, inplace=True)
test = test.drop(['id','breath_id','time_step'], axis=1)
#-------------cell------------------
COLS = train.columns.to_numpy()
#-------------cell------------------
# train = reduce_mem_usage(train)
# test = reduce_mem_usage(test)
#-------------cell------------------
RS = RobustScaler()
train = RS.fit_transform(train)
test = RS.transform(test)
#-------------cell------------------
train = train.reshape(-1, 80, train.shape[-1])
test = test.reshape(-1, 80, train.shape[-1])
#-------------cell------------------

In [ ]:
#Detect hardware, return appropriate distribution strategy
import tensorflow as tf
print(tf.version.VERSION)
#tf.get_logger().setLevel(logging.ERROR)
try: # detect TPU
    tpu = None
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError: # detect GPU(s) and enable mixed precision
    strategy = tf.distribute.MirroredStrategy() # works on GPU and multi-GPU
    policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
    tf.config.optimizer.set_jit(True) # XLA compilation
    tf.keras.mixed_precision.experimental.set_policy(policy)
    print('Mixed precision enabled')
print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
def plot_hist(hist):
    plt.plot(hist.history["loss"])
    plt.plot(hist.history["val_loss"])
    plt.title("model performance")
    plt.ylabel("mean_absolute_error")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()

In [ ]:
from tensorflow_addons.layers import MultiHeadAttention
from tensorflow.keras import backend as K
from tensorflow import keras 
class Time2Vec(tf.keras.layers.Layer):
    def __init__(self, kernel_size=1):
        super(Time2Vec, self).__init__(trainable=True, name='Time2VecLayer')
        self.k = kernel_size
    
    def build(self, input_shape):
        # trend
        self.wb = self.add_weight(name='wb',shape=(input_shape[1],),initializer='uniform',trainable=True)
        self.bb = self.add_weight(name='bb',shape=(input_shape[1],),initializer='uniform',trainable=True)
        # periodic
        self.wa = self.add_weight(name='wa',shape=(1, input_shape[1], self.k),initializer='uniform',trainable=True)
        self.ba = self.add_weight(name='ba',shape=(1, input_shape[1], self.k),initializer='uniform',trainable=True)
        super(Time2Vec, self).build(input_shape)
    
    def call(self, inputs, **kwargs):
        bias = self.wb * inputs + self.bb
        dp = K.dot(inputs, self.wa) + self.ba
        wgts = K.sin(dp) # or K.cos(.)

        ret = K.concatenate([K.expand_dims(bias, -1), wgts], -1)
        ret = K.reshape(ret, (-1, inputs.shape[1]*(self.k+1)))
        return ret
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1]*(self.k + 1))
#
class AttentionBlock(tf.keras.Model):
    def __init__(self, name='AttentionBlock', num_heads=2, head_size=128, ff_dim=None, dropout=0, **kwargs):
        super().__init__(name=name, **kwargs)

        if ff_dim is None:
            ff_dim = head_size

        self.attention = MultiHeadAttention(num_heads=num_heads, head_size=head_size, dropout=dropout)
        self.attention_dropout = keras.layers.Dropout(dropout)
        self.attention_norm = keras.layers.LayerNormalization(epsilon=1e-6)

        self.ff_conv1 = keras.layers.Conv1D(filters=ff_dim, kernel_size=1, activation='relu')
        # self.ff_conv2 at build()
        self.ff_dropout = keras.layers.Dropout(dropout)
        self.ff_norm = keras.layers.LayerNormalization(epsilon=1e-6)

    def build(self, input_shape):
        self.ff_conv2 = keras.layers.Conv1D(filters=input_shape[-1], kernel_size=1) 

    def call(self, inputs):
        x = self.attention([inputs, inputs])
        x = self.attention_dropout(x)
        x = self.attention_norm(inputs + x)

        x = self.ff_conv1(x)
        x = self.ff_conv2(x)
        x = self.ff_dropout(x)

        x = self.ff_norm(inputs + x)
        return x
#
class ModelTrunk(tf.keras.Model):
    def __init__(self, name='ModelTrunk', time2vec_dim=1, num_heads=2, head_size=128, ff_dim=None, num_layers=1, dropout=0, **kwargs):
        super().__init__(name=name, **kwargs)
        self.time2vec = Time2Vec(kernel_size=time2vec_dim)
        if ff_dim is None:
            ff_dim = head_size
        self.dropout = dropout
        self.attention_layers = [AttentionBlock(num_heads=num_heads, head_size=head_size, ff_dim=ff_dim, dropout=dropout) for _ in range(num_layers)]
        #self.pooling=tf.keras.layers.AveragePooling1D(pool_size=4,data_format='channels_first')
        self.final_layer = Sequential([
                Bidirectional(LSTM(256, return_sequences=True)),
                Bidirectional(LSTM(256, return_sequences=True)),
                Dense(128, activation='selu'),
                Dense(1)])
    def call(self, inputs):
        time_embedding = tf.keras.layers.TimeDistributed(self.time2vec)(inputs)
        #time_embedding=inputs
        #print(time_embedding.shape)
        x = K.concatenate([inputs, time_embedding], -1)
        for attention_layer in self.attention_layers:
            x = attention_layer(x)
        x=self.final_layer(x)
        return K.reshape(x, (-1, x.shape[1] * x.shape[2])) # flat vector of features out
t2v=ModelTrunk(ff_dim=256,num_heads=4,num_layers=2,)
x=tf.random.uniform((64,80,68))
x=t2v(x)
print(x.shape)

In [ ]:
def plot_lr():
    initial_learning_rate=0.001
    decay_rate=0.9#1e-5
    batch_size=512
    train_len=60360
    epoch_iter=int(train_len/batch_size)
    decay_steps=5#200*epoch_iter
    def decayed_learning_rate(step):
        return initial_learning_rate * pow(decay_rate , (step / decay_steps))
    lr_lst=[]
    for i in range(300):
        lr=decayed_learning_rate(i)
        lr_lst.append(lr)
    print(min(lr_lst))
    plt.plot(lr_lst)
    plt.show()
    plt.close()
plot_lr()

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=126)

test_preds = []
batch_size=128
for fold, (train_idx, test_idx) in enumerate(kf.split(train, targets)):
    print(f"****** fold: {fold+1} *******")
    X_train, X_valid = train[train_idx], train[test_idx]
    y_train, y_valid = targets[train_idx], targets[test_idx]
    #scheduler = tf.keras.optimizers.schedules.ExponentialDecay(1e-3, 200*((len(train)*0.8)/batch_size), 1e-5)
    scheduler = tf.keras.optimizers.schedules.ExponentialDecay(1e-3, 5, 0.9)
    es = EarlyStopping(monitor='val_loss',mode='min', patience=35, verbose=1,restore_best_weights=True)
    checkpoint_filepath = f"folds{fold}.tf"#f"folds{fold}.hdf5"
#     sv = keras.callbacks.ModelCheckpoint(
#             checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True,
#             save_weights_only=False, mode='auto', save_freq='epoch',
#             options=None
#         )
    with strategy.scope():
        model = ModelTrunk(ff_dim=256,num_heads=8,num_layers=2,)
        model.compile(optimizer="adam",loss = "mae")
    history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=250, batch_size = batch_size,
                        callbacks = [es,tf.keras.callbacks.LearningRateScheduler(scheduler)])
    test_preds.append(model.predict(test).squeeze().reshape(-1, 1).squeeze())
    #plot_hist(history)
    del X_train, X_valid, y_train, y_valid, model
    gc.collect()

In [ ]:
submission = pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')
submission["pressure"] =sum(test_preds)/len(test_preds)# test_preds[0]
submission.to_csv('submission.csv', index=False)
submission

In [ ]:
train_gf = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
all_pressure = sorted(train_gf.pressure.unique())
PRESSURE_MIN = all_pressure[0]
PRESSURE_MAX = all_pressure[-1]
PRESSURE_STEP = (all_pressure[1] - all_pressure[0])
submission["pressure"] = np.median(np.vstack(test_preds),axis=0)
submission["pressure"] =np.round( (submission.pressure - PRESSURE_MIN)/PRESSURE_STEP ) * PRESSURE_STEP + PRESSURE_MIN
submission.pressure = np.clip(submission.pressure, PRESSURE_MIN, PRESSURE_MAX)
pressure_unique = np.array(sorted(train_gf['pressure'].unique()))
submission['pressure'] = submission['pressure'].map(lambda x: pressure_unique[np.abs(pressure_unique-x).argmin()])
submission.to_csv('submission.csv', index = 0)
submission

In [ ]:
# import cupy, cudf, matplotlib.pyplot as plt
# train_gf = cudf.read_csv('../input/ventilator-pressure-prediction/train.csv')
# # plt.title('Histogram of Train Pressures',size=14)
# # plt.hist(train_gf.sample(100_000).pressure.to_array(),bins=100)
# # plt.show()
# print('Max pressure =',train_gf.pressure.max(), 'Min pressure =',train_gf.pressure.min())
# all_pressure = cupy.sort( train_gf.pressure.unique().values )
# print('The first 25 unique pressures...')
# PRESSURE_MIN = all_pressure[0].item()
# PRESSURE_MAX = all_pressure[-1].item()
# #
# PRESSURE_STEP = ( all_pressure[1] - all_pressure[0] ).item()

In [ ]:
# submission = pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')
# sub_1=pd.read_csv('../input/submission-list/submission172.csv')
# sub_2=pd.read_csv('../input/submission-list/submission_mean_LB157.csv')
# submission['pressure'] = (sub_1['pressure'].values*0.2)+(sub_2['pressure'].values*0.8)
# submission["pressure"]=np.round((submission.pressure-PRESSURE_MIN)/PRESSURE_STEP)*PRESSURE_STEP+PRESSURE_MIN
# submission.pressure=np.clip(submission.pressure, PRESSURE_MIN, PRESSURE_MAX)
# submission.to_csv('submission.csv', index=False)
# submission.head(5)